In [10]:
import cv2
import random
import numpy as np
import os

# Directories
arenas_dir = './arenas'
def placeShit(cutouts_dir):
    # Load the images
    cutouts = [cv2.imread(os.path.join(cutouts_dir, f), cv2.IMREAD_UNCHANGED) for f in os.listdir(cutouts_dir) if f.endswith('.png')]
    arenas = [cv2.imread(os.path.join(arenas_dir, f), cv2.IMREAD_UNCHANGED) for f in os.listdir(arenas_dir) if f.endswith('.png')]

    # Function to overlay transparent image
    def overlay_transparent(background, overlay, x, y):
        h, w, _ = overlay.shape  
        overlay_image = overlay[..., :3]  
        mask = overlay[..., 3:] / 255.0
        background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image
        return background

    img_to_gen = 1000  # number of images to generate

    for i in range(img_to_gen):
        background = arenas[i % len(arenas)].copy()  # use modulo to cycle through backgrounds
        background_h, background_w, _ = background.shape

        # initialize a list to hold the bounding boxes and labels
        bboxes = []
        labels = []

        # Select a random number of cutouts to use (between 'x' and 'y')
        num_cutouts = random.randint(0, 10)  # replace 'x' and 'y' with the actual range you want

        # Randomly select that number of cutouts from the list
        selected_cutouts = random.sample(cutouts, num_cutouts)

        for cutout in selected_cutouts:
            # Resize the cutout to be generally smaller
            cutout = cv2.resize(cutout, (cutout.shape[1] // 2, cutout.shape[0] // 2))

            cutout_h, cutout_w, _ = cutout.shape

            # Randomly place cutout on background
            x = random.randint(0, background_w - cutout_w)
            y = random.randint(0, background_h - cutout_h)
            
            background = overlay_transparent(background, cutout, x, y)

            # Define bounding box in YOLO format (center_x, center_y, width, height) normalized by the image width and height
            bbox = [(x + cutout_w / 2) / background_w, (y + cutout_h / 2) / background_h, cutout_w / background_w, cutout_h / background_h]
            bboxes.append(bbox)
            labels.append("Cassiopeia")
            
        # Save the transformed image
        cv2.imwrite(f'./output/images/{i}.jpg', background)

        # Save the bounding boxes in YOLO format
        with open(f'./output/labels/{i}.txt', 'w') as f:
            for bbox, label in zip(bboxes, labels):
                f.write(f'{label} {" ".join(map(str, bbox))}\n')
